In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import unidecode
import spacy

c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
top = pd.read_csv('top200.csv')

bottom = pd.read_csv('bottom200.csv')

In [4]:
def chorus(df):
    df['chorus'] = ''
    i = 0
    for morceau in df['lyrics']:
        split = str(morceau).split("Refrain")
        if len(split) == 1:
            split = str(morceau).split("Chorus")
        if len(split) == 1:
            df['chorus'][i] = ''
        else:
            chorus_and_co = split[1].split('] ')
            if len(chorus_and_co) == 1:
                df['chorus'][i] = ''
            else:
                chorus = chorus_and_co[1].split(' [')
                df['chorus'][i] = chorus[0]
        i+=1
    return df

In [5]:
top = chorus(top)
bottom = chorus(bottom)

C:\Users\bapti\AppData\Local\Temp\ipykernel_13596\1816682452.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = chorus[0]
C:\Users\bapti\AppData\Local\Temp\ipykernel_13596\1816682452.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = ''
C:\Users\bapti\AppData\Local\Temp\ipykernel_13596\1816682452.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = chorus[0]
C:\Users\bapti\AppDat

In [6]:
top['lyrics'].head()

0    [Paroles de "TOUT VA BIEN" ft. Naps & Ninho] [...
1    [Paroles de "FADE UP" ft. Hamza & SCH] [Intro ...
2    [Paroles de "DIE"] [Intro] La mala est gangx E...
3    [Paroles de "PETETE"] [Intro] Pew Grr Pew pew ...
4    [Paroles de "Time Time"] [Couplet 1 : Squeezie...
Name: lyrics, dtype: object

In [7]:
bottom['lyrics'].head()

0    [Paroles de "BUMPY"] [Couplet 1] Mourir du P38...
1    [Paroles de "VALLON DES AUFFES"] [Intro] HuFel...
2    [Paroles de "BISON"] [Intro] Breh breh [Couple...
3    [Paroles de "IMPACT"] [Intro] Gang gang gang g...
4    [Paroles de "BOSS"] [Intro] Nardey c'est chaud...
Name: lyrics, dtype: object

In [8]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

In [9]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    stop_words = stop_words.extend(['ouais', 'yeah', 'ca', 'eh', 'oh', 'han', 'ah', 'oh', 'hey', 'ooh'])
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [10]:
def add_lyrics_features(df):
    df['lyrics']=df['lyrics'].dropna()
    df['lyrics_nb_max'] = 0
    df['lyrics_nb_moy'] = 0
    i = 0
    while i < len(df['lyrics']):
        
        df['lyrics'][i] = str(df['lyrics'][i])

        if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
            df['lyrics'][i] = unidecode.unidecode(df['lyrics'][i])
            df['lyrics'][i] = re.sub(r'\[.*?\]', '', df['lyrics'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['lyrics'][i] = df['lyrics'][i].replace(c, ' ')
            df['lyrics'][i] = df['lyrics'][i].lower()
            df['lyrics'][i] = df['lyrics'][i].split(' ')
            df['lyrics'][i] = stop_words_english(df['lyrics'][i])
            df['lyrics'][i] = stop_words_french(df['lyrics'][i])

            if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
                d = {j:df['lyrics'][i].count(j) for j in df['lyrics'][i]}

                max = np.array(list(d.values())).max()
                mean = np.array(list(d.values())).mean()

                df['lyrics_nb_max'][i] = max
                df['lyrics_nb_moy'][i] = mean

        i +=1
    return df

In [11]:
def add_chorus_features(df):
    df['chorus']=df['chorus'].dropna()
    df['nb_words_chorus'] = 0
    df['nb_different_words_chorus'] = 0
    df['diff_duplicates_chorus'] = 0
    i = 0
    while i < len(df['chorus']):
        if (df['chorus'][i] != 'nan') and (len(df['chorus'][i]) > 0):
            df['chorus'][i] = unidecode.unidecode(df['chorus'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['chorus'][i] = df['chorus'][i].replace(c, ' ')
            df['chorus'][i] = df['chorus'][i].lower()
            df['chorus'][i] = df['chorus'][i].split(' ')
            df['chorus'][i] = stop_words_english(df['chorus'][i])
            df['chorus'][i] = stop_words_french(df['chorus'][i])

            nb_words = len(df['chorus'][i])
            without_duplicates = len(list(set(df['chorus'][i])))
            diff = nb_words - without_duplicates

            df['nb_words_chorus'][i] = nb_words
            df['nb_different_words_chorus'][i] = without_duplicates
            df['diff_duplicates_chorus'][i] = diff

        i +=1
    return df

In [12]:
#top = add_lyrics_features(top)
top = add_chorus_features(top)
top.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,lyrics,winter,spring,summer,autumn,id,chorus,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
0,TOUT VA BIEN (feat. Ninho & Naps),Alonzo,rap,1,0.660,0.720,3.0,-5.874,0.0,0.0753,...,"[Paroles de ""TOUT VA BIEN"" ft. Naps & Ninho] [...",0,1,1,1,6OZwia8loN0aPS0vTvsBjR,"[cherie, veut, yves, saint, lau, lau, donne, c...",45,30,15
1,FADE UP,ZEG P,autre,2,0.758,0.599,4.0,-5.990,0.0,0.0877,...,"[Paroles de ""FADE UP"" ft. Hamza & SCH] [Intro ...",0,0,1,1,4ZpIuzx91EAPK3VimONbfB,"[fade, fade, nan, fade, fade, nan, fade, fade,...",12,2,10
2,DIE,Gazo,rap,3,0.695,0.630,8.0,-7.160,0.0,0.0350,...,"[Paroles de ""DIE""] [Intro] La mala est gangx E...",0,0,1,1,3D29kjUyWxsT3jUUTtARVQ,"[crois, evite, alors, maille, maille, veut, sa...",34,19,15
3,PETETE,Gambi,rap,4,0.752,0.669,10.0,-9.817,0.0,0.2510,...,"[Paroles de ""PETETE""] [Intro] Pew Grr Pew pew ...",0,0,1,1,0z3bi63SNZ5ylyHOzb81Uq,"[cr, partout, bloc, okay, khabat, vite, pete, ...",38,27,11
4,Time Time,Trei Degete,autre,5,0.715,0.799,0.0,-6.447,0.0,0.0433,...,"[Paroles de ""Time Time""] [Couplet 1 : Squeezie...",1,0,0,1,5wKDPtbdggE1roeVp3UdXX,"[dansam, time, time, dansam, time, time, asa, ...",67,12,55


In [13]:
# bottom = add_lyrics_features(bottom)
bottom = add_chorus_features(bottom)
bottom.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,lyrics,winter,spring,summer,autumn,id,chorus,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
0,BUMPY,Alonzo,rap,1,0.713,0.820,5.0,-6.650,0.0,0.3670,...,"[Paroles de ""BUMPY""] [Couplet 1] Mourir du P38...",0,1,1,1,1D8lLfcmm2kpW7CTdhrFN9,"[avoir, vie, bumpy, coute, cher, cette, vida, ...",50,24,26
1,VALLON DES AUFFES,Alonzo,rap,2,0.644,0.798,5.0,-5.791,0.0,0.0961,...,"[Paroles de ""VALLON DES AUFFES""] [Intro] HuFel...",0,1,1,1,0mJHGKoDHfSkDxvzPbo6EV,"[pose, bas, chez, vis, nuit, fuis, jour, pah, ...",43,22,21
2,BISON,Alonzo,rap,3,0.838,0.693,4.0,-6.547,0.0,0.2610,...,"[Paroles de ""BISON""] [Intro] Breh breh [Couple...",0,1,1,1,2JkWkGL2OQHUpAdyRaH5a8,"[vrais, putas, tout, melange, melange, melange...",56,15,41
3,IMPACT,Gazo,rap,4,0.767,0.673,8.0,-5.843,0.0,0.2280,...,"[Paroles de ""IMPACT""] [Intro] Gang gang gang g...",0,0,1,1,1hlUHJgbdFdvZvG64a3W1g,"[veux, pe, mala, gang, gang, si, perds, impact...",34,30,4
4,BOSS,Gazo,rap,5,0.708,0.676,4.0,-8.016,0.0,0.1390,...,"[Paroles de ""BOSS""] [Intro] Nardey c'est chaud...",0,0,1,1,2K2hSiCZAwwlFvwTxVRqoi,"[revend, coke, bosse, vrais, machines, jamais,...",59,35,24


In [14]:
top.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,duration_ms,time_signature,winter,spring,summer,autumn,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
count,1320.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,...,1319.000000,1319.000000,1319.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000
mean,660.500000,0.721116,0.646079,5.282790,-7.003566,0.391964,0.175331,0.291488,0.007608,0.153380,...,123.661030,196318.349507,3.957544,0.373485,0.359848,0.319697,0.491667,28.550000,16.093182,12.456818
std,381.195488,0.128060,0.147248,3.572734,2.246327,0.488374,0.137636,0.223102,0.058262,0.104487,...,25.372279,48481.258432,0.308728,0.483913,0.480138,0.466536,0.500120,21.964178,13.279110,14.308513
min,1.000000,0.234000,0.120000,0.000000,-21.189000,0.000000,0.025800,0.000307,0.000000,0.023200,...,45.130000,32107.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,330.750000,0.641500,0.549000,2.000000,-8.147500,0.000000,0.056800,0.110500,0.000000,0.092800,...,104.128500,167453.500000,4.000000,0.000000,0.000000,0.000000,0.000000,12.750000,6.000000,1.000000
50%,660.500000,0.742000,0.659000,6.000000,-6.670000,0.000000,0.129000,0.247000,0.000000,0.115000,...,125.135000,190373.000000,4.000000,0.000000,0.000000,0.000000,0.000000,26.500000,14.000000,8.000000
75%,990.250000,0.813500,0.754000,8.000000,-5.531500,1.000000,0.271500,0.420000,0.000043,0.173000,...,140.052500,215093.500000,4.000000,1.000000,1.000000,1.000000,1.000000,43.000000,24.000000,18.000000
max,1320.000000,0.973000,0.965000,11.000000,-2.018000,1.000000,0.828000,0.994000,0.956000,0.812000,...,201.898000,515387.000000,5.000000,1.000000,1.000000,1.000000,1.000000,165.000000,119.000000,139.000000


In [15]:
bottom.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,duration_ms,time_signature,winter,spring,summer,autumn,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
count,1053.00000,1051.000000,1051.000000,1051.00000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,...,1051.000000,1053.000000,1051.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000
mean,527.00000,0.667426,0.592365,5.16746,-7.984419,0.472883,0.165137,0.335532,0.034373,0.162281,...,121.102184,198573.805318,3.914367,0.484330,0.450142,0.421652,0.592593,21.449193,11.733143,9.716049
std,304.11922,0.166181,0.179246,3.57485,3.101373,0.499502,0.152667,0.268749,0.152795,0.117524,...,27.957843,56874.727095,0.427944,0.499992,0.497744,0.494058,0.491585,19.619250,11.281399,12.227101
min,1.00000,0.063300,0.034600,0.00000,-25.521000,0.000000,0.024100,0.000030,0.000000,0.026100,...,61.466000,60123.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,264.00000,0.567000,0.501000,2.00000,-9.037500,0.000000,0.047250,0.111000,0.000000,0.096600,...,98.996000,167256.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,527.00000,0.705000,0.615000,5.00000,-7.265000,0.000000,0.101000,0.266000,0.000000,0.116000,...,122.000000,191828.000000,4.000000,0.000000,0.000000,0.000000,1.000000,18.000000,10.000000,4.000000
75%,790.00000,0.788000,0.713000,8.00000,-6.088500,1.000000,0.256000,0.511000,0.000107,0.186000,...,139.966000,219120.000000,4.000000,1.000000,1.000000,1.000000,1.000000,34.000000,18.000000,17.000000
max,1053.00000,0.973000,0.968000,11.00000,-2.021000,1.000000,0.956000,0.992000,0.965000,0.848000,...,202.008000,966360.000000,5.000000,1.000000,1.000000,1.000000,1.000000,100.000000,61.000000,82.000000


In [16]:
test_on_refrain = top['chorus']

In [17]:
data_words = list(test_on_refrain)

In [18]:
data_wordss = [words for words in data_words if words != "" and words != [] and words != 'nan']

In [19]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_wordss)
# Create Corpus
texts = data_wordss
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 1), (9, 1), (10, 1), (11, 6), (12, 2), (13, 2), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1)]


In [20]:
print(len(id2word))

6449


In [21]:
tot = 0
i = 1
for c in corpus:
    tot += len(c)
    i+=1
print(tot/i)

19.435498627630373


In [22]:
from pprint import pprint
import gensim
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

: 

In [69]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.034853  0.061895       1        1  13.074225
9     -0.005602 -0.035342       2        1  12.854761
4      0.020906  0.090938       3        1  12.608178
2      0.043703  0.025092       4        1  10.539156
3     -0.127014  0.052042       5        1   9.931696
0      0.034876 -0.113310       6        1   9.530427
6     -0.028745 -0.068862       7        1   8.422645
8      0.044111 -0.000968       8        1   7.951742
5     -0.096079 -0.023872       9        1   7.701223
7      0.078990  0.012388      10        1   7.385948, topic_info=      Term        Freq       Total Category  logprob  loglift
117     ah  347.000000  347.000000  Default  30.0000  30.0000
129  ouais  328.000000  328.000000  Default  29.0000  29.0000
79      oh  501.000000  501.000000  Default  28.0000  28.0000
247   yeah  320.000000  320.000000  Default  27.0000  27.0000
242    ouh  157.000000  157.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
206  comme   11.611479  304.703108  Topic10  -5.4795  -0.6618
25    tout   11.581626  314.749090  Topic10  -5.4820  -0.6968
219     si   10.544756  218.536127  Topic10  -5.5758  -0.4257
12    fais    9.499881  211.833743  Topic10  -5.6802  -0.4989
223    vie    8.479256  126.183453  Topic10  -5.7938  -0.0945

[740 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3414      2  0.133767       "a
3414      6  0.802601       "a
3914      8  0.761424      "je
1689      5  0.712755  "laisse
2406      5  0.712674  "likes"
...     ...       ...      ...
444       8  0.021427     zone
444       9  0.064282     zone
4816      1  0.615973   zoulou
4816      2  0.087996   zoulou
4816      3  0.263989   zoulou

[1793 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 5, 3, 4, 1, 7, 9, 6, 8])

In [70]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data_wordss):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_wordss)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

C:\Users\bapti\AppData\Local\Temp\ipykernel_7544\3058185832.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9,0.9804,"tout, ah, oh, han, plus, meme, si, fais, quand, deux","[cherie, veut, yves, saint, lau, lau, donne, coeur, fais, bobo, oui, meme, tarpin, zehef, fais, ..."
1,1,9,0.9308,"tout, ah, oh, han, plus, meme, si, fais, quand, deux","[fade, fade, nan, fade, fade, nan, fade, fade, nan, fade, fade, nan]"
2,2,2,0.9743,"ca, fais, tout, yeah, oh, sous, veux, vie, quand, bebe","[crois, evite, alors, maille, maille, veut, savoir, quel, bail, quel, bail, dis, veux, voit, voi..."
3,3,5,0.9769,"yeah, faire, ouais, bo, sais, ola, teh, ca, plus, comme","[cr, partout, bloc, okay, khabat, vite, pete, okay, bebe, donne, ca, bien, okay, bebe, fait, ca,..."
4,4,4,0.9868,"oh, ca, yeah, si, quand, fait, ouh, fais, ya, vie","[dansam, time, time, dansam, time, time, asa, ca, bye, bye, plecam, suntem, gata, calatorim, tim..."
5,5,5,0.9743,"yeah, faire, ouais, bo, sais, ola, teh, ca, plus, comme","[putain, merde, plan, gate, full, ksaara, meme, perte, plus, tchop, mehlich, va, patter, bitchie..."
6,6,3,0.9526,"ouais, ouh, comme, rien, ca, trop, plus, ah, oh, yeah","[poursuites, rafales, bon, shit, 200, sangs, periph, mythos, poukies, montent, caisses, ches, lo..."
7,7,9,0.9678,"tout, ah, oh, han, plus, meme, si, fais, quand, deux","[pete, trois, fois, filtre, narguile, trop, bouteilles, enquillees, sais, meme, plus, heure, pou..."
8,8,0,0.8875,"tout, sais, know, mmh, ca, yeah, comme, oh, let, gang","[world, us, know, world, us, know, know]"
9,9,4,0.9346,"oh, ca, yeah, si, quand, fait, ouh, fais, ya, vie","[toujours, reve, etre, top, sourd, face, jalousie, jalousie, fait, benef, confort, bon, dieu, ad..."


In [71]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.9889,"tout, sais, know, mmh, ca, yeah, comme, oh, let, gang","[dit, ""mmh, mmh, okay, mmh, da, ouais, tout"", okay, okay, dit, ""mmh, mmh, okay, mmh, da, prends,..."
1,1,0.9916,"ca, comme, ooh, fait, hey, eh, ya, faire, yeah, han","[va, rouler, paris, toute, nuit, faut, oublie, faut, oublie, ennuies, faut, rester, solide, rest..."
2,2,0.9886,"ca, fais, tout, yeah, oh, sous, veux, vie, quand, bebe","[dit, ""longue, vie, bebe"", longue, vie, bebe, longue, vie, bebe, longue, vie, bebe, dit, ""longue..."
3,3,0.9894,"ouais, ouh, comme, rien, ca, trop, plus, ah, oh, yeah","[let, get, crunk, fun, dancery, got, ya, open, ya, floatin, gots, dance, need, hateration, holle..."
4,4,0.9943,"oh, ca, yeah, si, quand, fait, ouh, fais, ya, vie","[og, og, og, og, og, og, og, protege, gang, lobby, protege, gang, lobby, faudra, kich, quitter, ..."
5,5,0.9924,"yeah, faire, ouais, bo, sais, ola, teh, ca, plus, comme","[negros, font, fraudes, negros, fraudes, scam, scam, scam, scam, si, touche, cibles, frole, grr,..."
6,6,0.9912,"ah, ca, plus, bien, ouais, quand, comme, shayo, fait, si","[ca, va, porte, encore, intacte, matin, juge, fait, ufc, quand, belier, fait, karate, bras, cass..."
7,7,0.9875,"fait, plus, faut, moissanite, vais, lo, rien, oh, yah, gang","[oublie, meme, si, ja, pardonne, si, ja, pardonne, route, longue, vu, bateau, couler, ah, ca, pe..."
8,8,0.9873,"eh, comme, si, hey, oh, veux, fait, ca, tout, plus","[fil, coupe, pendu, sous, combine, eh, eh, eh, eh, rhum, clope, rancune, tout, combine, eh, eh, ..."
9,9,0.9898,"tout, ah, oh, han, plus, meme, si, fais, quand, deux","[ouais, si, gros, boule, bouge, boule, agite, foule, tout, monde, devient, fou, si, gros, boule,..."


In [130]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

NameError: name 'stop_words' is not defined